# Setting environment

In [1]:
import platform

print(f"Python version: {platform.python_version()}")
assert platform.python_version_tuple() >= ("3", "6")

# import numpy as np
# import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

Python version: 3.8.9


In [2]:
# Setup plots
%matplotlib inline
plt.rcParams["figure.figsize"] = 10, 8
%config InlineBackend.figure_format = 'retina'
sns.set()
%load_ext tensorboard

In [3]:
import torch

print(f"PyTorch version: {torch.__version__}")

import torch.nn as nn
# from torch.utils.tensorboard import SummaryWriter

PyTorch version: 1.10.1


# Building the model

In [9]:
# noinspection PyTypeChecker

class Generator(nn.Module):
    def __init__(self, in_channels):
        super(Generator, self).__init__()
        self.num_gpu = num_gpu
        self.generator = nn.Sequential(
            nn.ConvTranspose2d(in_channels, 64, 7, 1, 0, bias=False),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 32, 4, 2, 1, bias=False),
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 1, 4, 2, 1, bias=False),
            nn.Tanh(), # Only the last layer uses a tanh activations
        )

    def forward(self, x):
        return self.generator(x)

In [7]:
# noinspection PyTypeChecker

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.discriminator = nn.Sequential(
            nn.Conv2d(1, 32, 4, 2, 1, bias=False),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Conv2d(32, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Conv2d(64, 1, 7, 1, 0, bias=False),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.discriminator(x)

In [8]:

num_gpu = 1
num_epochs = 1 # to change later
batch_size = 32
device = torch.device('cuda:0' if (torch.cuda.is_available() and num_gpu > 0) else 'cpu')

generator = Generator(1).to(device)
discriminator = Discriminator().to(device)

# Handle multi gpu
if (device.type == 'cuda') and (num_gpu > 1):
    generator = nn.DataParallel(generator, list(range(num_gpu)))
    discriminator = nn.DataParallel(discriminator, list(range(num_gpu)))


print(generator)
print(discriminator)

Generator(
  (generator): Sequential(
    (0): ConvTranspose2d(1, 64, kernel_size=(7, 7), stride=(1, 1), bias=False)
    (1): ReLU(inplace=True)
    (2): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): ReLU(inplace=True)
    (4): ConvTranspose2d(32, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (5): Tanh()
  )
)
Discriminator(
  (discriminator): Sequential(
    (0): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): Conv2d(64, 1, kernel_size=(7, 7), stride=(1, 1), bias=False)
    (5): Sigmoid()
  )
)


In [ ]:
generator_optimizer = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
criterion = nn.BCELoss()

generator_loss = []
discriminator_loss = []

real_label = 1
fake_label = 0
z = 100

In [ ]:
dataset_batches = None

In [ ]:
for epoch in range(num_epochs):
    for i, data in enumerate(dataset_batches, 0):
        ###  Discriminator training

        # reset the gradients
        discriminator.zero_grad()

        # transfer the images to the specified device (required for
        # training on a gpu)
        real_images = data[0].to(device)
        labels = torch.full((batch_size,), real_label, device=device)

        # Forward pass the real batch through discriminator
        output = discriminator(real_images).view(-1)
        prob_D = output.mean().item()
        errD_real = criterion(output, labels)
        errD_real_mean = output.mean().item()

        # generate fake images
        noise_batch = torch.randn(batch_size, z, 1, 1, device=device)
        fake_images = generator(noise_batch).to(device)
        labels = torch.full((batch_size,), fake_label, device=device)

        # Forward pass the fake batch through discriminator
        output = discriminator(fake_images.detach()).view(-1)
        errD_fake = criterion(output, labels)
        errD_fake_mean = output.mean().item()

        errD = errD_real + errD_fake
        errD.backward()

        # Make a gradient step
        discriminator_optimizer.step()

        ####  Generator training

        generator.zero_grad()

        # create another batch of fakes for the generator
        noise_batch = torch.randn(batch_size, z, 1, 1, device=device)
        fake_images = generator(noise_batch).to(device)
        labels = torch.full((batch_size,), real_label, device=device)

        # Forward pass the fake batch through generator
        output = discriminator(fake_images).view(-1)
        prob_G = output.mean().item()
        errG = criterion(output, labels)
        errG_mean = output.mean().item()

        errG.backward()
        generator_optimizer.step()

        # Record the training stats
        if i % 50 == 0:
            print(f"[{epoch}/{num_epochs}][{i}/{len(dataset_batches)}]\tLoss_D: {errD.item()}\tLoss_G: {errG.item()}\tProb D: {prob_D}\tProb G: {prob_G}")

        # Save the loss profiles for plotting later
        discriminator_loss.append(errD.item())
        generator_loss.append(errG.item())

        epoch += 1